In [1]:
import pandas as pd
import numpy as np

In [23]:
def get_entropy(px):
    if px == 0:
        return 0;
    return px * np.log2(px)

In [3]:
df_17 = pd.read_csv('../PBL/dataset/NHIS_2017_2018_100m/NHIS_OPEN_GJ_2017_100.csv', encoding='euc-kr')
df_18 = pd.read_csv('../PBL/dataset/NHIS_2017_2018_100m/NHIS_OPEN_GJ_2018_100.csv')

In [4]:
valid_17 = df_17.loc[df_17['식전혈당(공복혈당)'] <= 300]
valid_18 = df_18.loc[df_18['식전혈당(공복혈당)'] <= 300]

In [34]:
valid_df = valid_17.append(valid_18)

valid_df["성별코드"].unique()
# valid_df["연령대코드(5세단위)"].unique()
# valid_df["신장(5Cm단위)"].unique()
# valid_df["체중(5Kg단위)"].unique()

# valid_df.dropna(subset=["허리둘레"], inplace=True)
# valid_df = valid_df.loc[valid_df["허리둘레"] >= 35]
# valid_df = valid_df.loc[valid_df["허리둘레"] <= 111.76]

array([1, 2])

In [35]:
col_num = valid_df.columns.size
row_num = valid_df.size / col_num

In [36]:
idx = pd.RangeIndex(start=0, stop=row_num)
valid_df.index=idx

In [37]:
confirmed_person = valid_df.loc[valid_df["식전혈당(공복혈당)"] >= 126]
confirmed_mask = valid_df["식전혈당(공복혈당)"] >= 126
confirmed_size = confirmed_person.size / col_num

In [38]:
unconfirmed_person = valid_df.loc[valid_df["식전혈당(공복혈당)"] < 126]
unconfirmed_size = row_num - confirmed_size

In [39]:
confirmed_px_root = confirmed_size / row_num
unconfirmed_px_root = 1 - confirmed_px_root

In [40]:
entropy_root = -1 * (get_entropy(confirmed_px_root) + get_entropy(unconfirmed_px_root))

In [41]:
print(entropy_root)

0.390673901150853


In [42]:
def getIG(col_name, value_list):
    sum_entropy = 0
    for val in value_list:
        df = valid_df.loc[valid_df[col_name] == val]
        df_size = df.size / col_num
        
        Y_df = df.loc[df["식전혈당(공복혈당)"] >= 126]
        
        Y_df_size = Y_df.size / col_num
        
        Y_px = Y_df_size / df_size
        N_px = 1 - Y_px
        rtn = -1 * (get_entropy(Y_px) + get_entropy(N_px))
        sum_entropy += rtn * (df_size / row_num)
        
    return entropy_root - sum_entropy


def getRangeIG(col_name, range_list):
    sum_entropy = 0
    prev_ran = 0
    for ran in range_list:
        df = valid_df.loc[valid_df[col_name] > prev_ran]
        df = df.loc[df[col_name] <= ran] 
        df_size = df.size / col_num
        
        Y_df = df.loc[df["식전혈당(공복혈당)"] >= 126]
        
        Y_df_size = Y_df.size / col_num
        
        Y_px = Y_df_size / df_size
        N_px = 1 - Y_px
        
        rtn = -1 * (get_entropy(Y_px) + get_entropy(N_px))
        sum_entropy += rtn * (df_size / row_num)
        prev_ran = ran
    
    return entropy_root - sum_entropy
        


In [43]:
print(getIG("성별코드", valid_df["성별코드"].unique()))

0.003918456695817585


In [69]:
print(getIG("연령대코드(5세단위)", valid_df["연령대코드(5세단위)"].unique()))

-11.745756122504782


In [98]:
# print(valid_df["흡연상태"].unique())
print(valid_df["흡연상태"].value_counts())
print(getIG("흡연상태", valid_df["흡연상태"].unique()))

1.0    1209524
3.0     428063
2.0     353015
Name: 흡연상태, dtype: int64
0.003135562879630127


In [79]:
print(getIG("음주여부", valid_df["음주여부"].unique()))

# valid_df.loc[valid_df["음주여부"] == (1 or 0)]
# print(valid_df["음주여부"].unique())

0.00911059993264457


In [156]:
waist_list = [66.04, 71.12, 78.74, 83.82, 104.14, 106.68, 111.76]

print(getRangeIG("허리둘레", waist_list))

0.0187891697088981


In [26]:
# valid_df["체중(5Kg단위)"].unique()
getIG("체중(5Kg단위)",valid_df["체중(5Kg단위)"].unique())

0.00569303427192025